In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso

### Regresión simple (y = mpg)

In [4]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
df = df.drop(columns=["model"])
X = df.drop(columns=["mpg"])
y = df["mpg"]
X = X.astype(float)
y= y.astype(float)

In [5]:
# Regresión 
X_ols=sm.add_constant(X)
ols = sm.OLS(y, X_ols)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.869
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     13.93
Date:                Thu, 18 Sep 2025   Prob (F-statistic):           3.79e-07
Time:                        22:13:00   Log-Likelihood:                -69.855
No. Observations:                  32   AIC:                             161.7
Df Residuals:                      21   BIC:                             177.8
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.3034     18.718      0.657      0.518     -26.623      51.229
cyl           -0.1114      1.045     -0.107      0.916      -2.285       2.062
disp           0.0133      0.018      0.747      0.463      -0.024       0.050
hp            -0.0215      0.022     -0.987      0.335      -0.067       0.024
drat           0.7871      1.635      0.481      0.635      -2.614       4.188
wt            -3.7153      1.894     -1.961      0.063      -7.655       0.224
qsec           0.8210      0.731      1.123      0.274      -0.699       2.341
vs             0.3178      2.105      0.151      0.881      -4.059       4.694
am             2.5202      2.057      1.225      0.234      -1.757       6.797
gear           0.6554      1.493      0.439      0.665      -2.450       3.761
carb          -0.1994      0.829     -0.241      0.812      -1.923       1.524
==============================================================================
Omnibus:                        1.907   Durbin-Watson:                   1.861
Prob(Omnibus):                  0.385   Jarque-Bera (JB):                1.747
Skew:                           0.521   Prob(JB):                        0.418
Kurtosis:                       2.526   Cond. No.                     1.22e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

const (12.30) → valor base del mpg cuando todas las demás variables son 0. 

cyl (-0.11) → más cilindros tienden a reducir un poco el rendimiento, aunque aquí el efecto es muy pequeño y no significativo.

disp (+0.013) → mayor desplazamiento del motor se asocia con un ligero aumento del mpg, lo cual es raro (esperaríamos lo contrario). Esto pasa porque está correlacionado con otras variables.

hp (-0.021) → más caballos de fuerza reducen el mpg, como se espera (motores más potentes consumen más).

drat (+0.79) → una mayor relación de eje se asocia con más mpg, aunque no es claro en este modelo.

wt (-3.72) → autos más pesados consumen más: cada 1000 libras extra reducen ~3.7 mpg. Este efecto sí es fuerte y consistente.

qsec (+0.82) → un mayor tiempo en el cuarto de milla (autos más lentos en aceleración) se relaciona con mejor rendimiento. Tiene lógica: motores menos deportivos tienden a ser más eficientes.

vs (+0.31) → motores en línea (vs=1) darían un poco más de mpg que los en V (vs=0), pero el efecto es muy débil aquí.

am (+2.52) → transmisión manual tiende a dar más mpg que la automática (~2.5 mpg extra). 

gear (+0.65) → más marchas en la transmisión tienden a mejorar ligeramente el mpg.

carb (-0.20) → más carburadores se asocian con menor eficiencia, aunque en este modelo el efecto es mínimo.

El modelo explica alrededor del 87% de la variabilidad en el consumo (mpg). Como conjunto funciona muy bien para explicar el rendimiento de los autos, pero la mayoría de las variables están demasiado correlacionadas entre sí. El peso del coche es el predictor más importante y con más evidencia estadística de afectar al consumo, mientras que el resto no se distingue claramente.

### Train - Test

In [32]:
Xtrain, Xtest, ytrain, ytest=train_test_split(X,y,train_size=.4,random_state=137)

In [33]:
scaler = StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtest_scaled  = scaler.transform(Xtest)

In [34]:
X_ols=sm.add_constant(Xtrain_scaled)
ols = sm.OLS(ytrain, X_ols)
results = ols.fit()
results.summary()

C:\Users\Michelle Trigo\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.891
Method:                 Least Squares   F-statistic:                     9.981
Date:                Thu, 18 Sep 2025   Prob (F-statistic):              0.242
Time:                        17:06:51   Log-Likelihood:                -9.9811
No. Observations:                  12   AIC:                             41.96
Df Residuals:                       1   BIC:                             47.30
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.4333      0.556     31.361      0.020      10.370      24.497
x1            40.7571     54.010      0.755      0.588    -645.499     727.013
x2            -2.4635      3.021     -0.815      0.565     -40.854      35.927
x3           -10.1581     19.484     -0.521      0.694    -257.727     237.411
x4            14.1135     13.469      1.048      0.485    -157.022     185.249
x5            -0.8005      3.106     -0.258      0.839     -40.266      38.665
x6             1.5727      3.475      0.453      0.729     -42.583      45.729
x7            32.4926     41.262      0.787      0.575    -491.795     556.780
x8            36.1394     42.669      0.847      0.553    -506.022     578.301
x9           -38.3551     43.625     -0.879      0.541    -592.668     515.958
x10           11.9707     17.482      0.685      0.618    -210.157     234.098
==============================================================================
Omnibus:                        0.259   Durbin-Watson:                   2.354
Prob(Omnibus):                  0.879   Jarque-Bera (JB):                0.413
Skew:                           0.212   Prob(JB):                        0.813
Kurtosis:                       2.196   Cond. No.                         382.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
y_pred_train = results.predict(sm.add_constant(Xtrain_scaled))
y_pred_test  = results.predict(sm.add_constant(Xtest_scaled))

print("R2 train:", r2_score(ytrain, y_pred_train))
print("R2 test:",  r2_score(ytest, y_pred_test))

R2 train: 0.9900805114414537
R2 test: -54.07178520418498


El modelo ajusta casi perfecto a los datos de entrenamiento (R2=0.99), pero al probarlo en datos nuevos su desempeño cae drásticamente (R2=-54), lo que indica un fuerte sobreajuste: con tan pocos datos de entrenamiento y muchas variables, la regresión prácticamente memorizó la muestra y no generaliza. En conclusión, aunque parece muy bueno en train, en realidad no sirve para predecir en test

### Ridge

In [36]:
# Lambda: .7
ridge = Ridge(alpha=.7)  
ridge.fit(Xtrain_scaled, ytrain)
y_pred_ridge_train = ridge.predict(Xtrain_scaled)
y_pred_ridge_test = ridge.predict(Xtest_scaled)


r2_train = r2_score(ytrain, y_pred_ridge_train)
r2_test = r2_score(ytest, y_pred_ridge_test)


print("Intercepto",ridge.intercept_,"Coeficientes",ridge.coef_, "R2 Train",r2_train,"R2 Test",r2_test )

Intercepto 17.433333333333334 Coeficientes [-1.91180995 -1.47680791  1.49298425  1.89807932 -0.66063204  1.45742956
 -0.05266481  1.49531023 -1.5340845  -0.82839949] R2 Train 0.9490586030413806 R2 Test 0.5420583567892368


In [38]:
# Lambda: .5
ridge = Ridge(alpha=.5)  
ridge.fit(Xtrain_scaled, ytrain)
y_pred_ridge_train = ridge.predict(Xtrain_scaled)
y_pred_ridge_test = ridge.predict(Xtest_scaled)


r2_train = r2_score(ytrain, y_pred_ridge_train)
r2_test = r2_score(ytest, y_pred_ridge_test)


print("Intercepto",ridge.intercept_,"Coeficientes",ridge.coef_, "R2 Train",r2_train,"R2 Test",r2_test )

Intercepto 17.43333333333333 Coeficientes [-1.99332987 -1.69676335  1.98249196  2.12288228 -0.58317461  1.64679902
 -0.11551529  1.58200655 -1.84690932 -0.91193747] R2 Train 0.9579016413911189 R2 Test 0.48980134996965086


In [39]:
# Lambda: .9
ridge = Ridge(alpha=.9)  
ridge.fit(Xtrain_scaled, ytrain)
y_pred_ridge_train = ridge.predict(Xtrain_scaled)
y_pred_ridge_test = ridge.predict(Xtest_scaled)


r2_train = r2_score(ytrain, y_pred_ridge_train)
r2_test = r2_score(ytest, y_pred_ridge_test)


print("Intercepto",ridge.intercept_,"Coeficientes",ridge.coef_, "R2 Train",r2_train,"R2 Test",r2_test )

Intercepto 17.433333333333334 Coeficientes [-1.83753463 -1.33741463  1.14933154  1.73728638 -0.70486346  1.3219195
 -0.0027361   1.42934836 -1.30288694 -0.77612781] R2 Train 0.9418178078071817 R2 Test 0.5773542854640781


La regresión Ridge mejora la OLS normal porque, aunque ajusta un poco peor los datos de entrenamiento, logra predecir mucho mejor los datos nuevos, evitando el sobreajuste. El valor de lambda controla cuánto se penalizan los coeficientes; un lambda bajo deja los coeficientes grandes y ajusta muy bien en train pero mal en test, mientras que un lambda alto reduce los coeficientes, baja un poco el ajuste en train pero mejora la predicción en test. El objetivo es encontrar un lambda intermedio que logre un buen equilibrio entre ajustar los datos y generalizar a nuevos casos.

**___________________________________________________________________________________________________________________________**

### Regresión simple (y = qsec)

In [13]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
df = df.drop(columns=["model"])
X = df.drop(columns=["qsec"])
y = df["qsec"]
X = X.astype(float)
y= y.astype(float)

In [14]:
# Regresión 
X_ols=sm.add_constant(X)
ols = sm.OLS(y, X_ols)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   qsec   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     14.66
Date:                Thu, 18 Sep 2025   Prob (F-statistic):           2.44e-07
Time:                        22:34:26   Log-Likelihood:                -30.242
No. Observations:                  32   AIC:                             82.48
Df Residuals:                      21   BIC:                             98.61
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.7762      3.876      4.586      0.000       9.716      25.837
mpg            0.0690      0.061      1.123      0.274      -0.059       0.197
cyl           -0.3627      0.293     -1.239      0.229      -0.971       0.246
disp          -0.0075      0.005     -1.505      0.147      -0.018       0.003
hp            -0.0016      0.006     -0.242      0.811      -0.015       0.012
drat          -0.1311      0.476     -0.275      0.786      -1.121       0.859
wt             1.4963      0.500      2.990      0.007       0.456       2.537
vs             0.9700      0.573      1.694      0.105      -0.221       2.161
am            -0.9012      0.585     -1.540      0.139      -2.118       0.316
gear          -0.2013      0.433     -0.465      0.647      -1.101       0.699
carb          -0.2736      0.233     -1.174      0.254      -0.758       0.211
==============================================================================
Omnibus:                       21.069   Durbin-Watson:                   2.573
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               38.291
Skew:                           1.470   Prob(JB):                     4.84e-09
Kurtosis:                       7.481   Cond. No.                     8.77e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.77e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

El modelo explica alrededor del 87% de la variabilidad de qsec (tiempo en cuarto de milla).
const (17.78) → valor base de qsec cuando todas las demás variables son cero.

mpg (+0.069) → autos más eficientes tienden a tener un ligero aumento en qsec (autos más lentos en aceleración), aunque el efecto es muy pequeño y no significativo.

cyl (-0.36) → más cilindros tienden a reducir qsec (autos más potentes aceleran más rápido), efecto pequeño.

disp (-0.0075) → mayor desplazamiento del motor se asocia con un tiempo de cuarto de milla ligeramente menor (más rápido).

hp (-0.0016) → más caballos de fuerza reducen un poco qsec (más rápido), efecto muy pequeño.

drat (-0.13) → relación de eje mayor tiende a reducir ligeramente qsec, efecto débil.

wt (+1.50) → autos más pesados tardan más en recorrer el cuarto de milla, efecto fuerte y significativo.

vs (+0.97) → motor en línea (vs=1) aumenta qsec (autos más lentos), efecto moderado.

am (-0.90) → transmisión manual tiende a hacer los autos más rápidos (reduce qsec).

gear (-0.20) → más marchas tienden a reducir ligeramente qsec, efecto pequeño.

carb (-0.27) → más carburadores tienden a reducir qsec (autos más deportivos), efecto pequeño.

   ### Train - Test

In [15]:
Xtrain, Xtest, ytrain, ytest=train_test_split(X,y,train_size=.4,random_state=137)

In [16]:
scaler = StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtest_scaled  = scaler.transform(Xtest)

In [19]:
X_ols=sm.add_constant(Xtrain_scaled)
ols = sm.OLS(ytrain, X_ols)
results = ols.fit()
results.summary()

C:\Users\Michelle Trigo\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1736: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   qsec   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     4.609
Date:                Thu, 18 Sep 2025   Prob (F-statistic):              0.349
Time:                        22:35:08   Log-Likelihood:               -0.25847
No. Observations:                  12   AIC:                             22.52
Df Residuals:                       1   BIC:                             27.85
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         17.3733      0.247     70.269      0.009      14.232      20.515
x1             1.0235      2.262      0.453      0.729     -27.713      29.760
x2            -0.3081     30.092     -0.010      0.993    -382.663     382.047
x3             0.2789      1.711      0.163      0.897     -21.465      22.023
x4            -1.0127      9.720     -0.104      0.934    -124.519     122.494
x5            -1.0356      8.615     -0.120      0.924    -110.497     108.426
x6             0.5976      1.295      0.461      0.725     -15.861      17.057
x7             0.2129     23.358      0.009      0.994    -296.578     297.004
x8            -0.3877     24.867     -0.016      0.990    -316.349     315.574
x9             0.6755     25.827      0.026      0.983    -327.485     328.836
x10           -0.1562      9.422     -0.017      0.989    -119.876     119.563
==============================================================================
Omnibus:                       12.359   Durbin-Watson:                   2.150
Prob(Omnibus):                  0.002   Jarque-Bera (JB):                7.063
Skew:                           1.532   Prob(JB):                       0.0293
Kurtosis:                       5.178   Cond. No.                         517.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
y_pred_train = results.predict(sm.add_constant(Xtrain_scaled))
y_pred_test  = results.predict(sm.add_constant(Xtest_scaled))

print("R2 train:", r2_score(ytrain, y_pred_train))
print("R2 test:",  r2_score(ytest, y_pred_test))

R2 train: 0.978762192523052
R2 test: 0.37610389619995843


El modelo ajusta muy bien los datos de entrenamiento (R2 = 0.98), pero su desempeño en prueba cae mucho (R2 = 0.38), lo que indica sobreajuste con solo 12 datos de entrenamiento y muchas variables.

### Ridge

In [46]:
# Lambda: .7
ridge = Ridge(alpha=.7)  
ridge.fit(Xtrain_scaled, ytrain)
y_pred_ridge_train = ridge.predict(Xtrain_scaled)
y_pred_ridge_test = ridge.predict(Xtest_scaled)


r2_train = r2_score(ytrain, y_pred_ridge_train)
r2_test = r2_score(ytest, y_pred_ridge_test)


print("Intercepto",ridge.intercept_,"Coeficientes",ridge.coef_, "R2 Train",r2_train,"R2 Test",r2_test )

Intercepto 17.433333333333334 Coeficientes [-1.91180995 -1.47680791  1.49298425  1.89807932 -0.66063204  1.45742956
 -0.05266481  1.49531023 -1.5340845  -0.82839949] R2 Train 0.9490586030413806 R2 Test 0.5420583567892368


In [49]:
# Lambda: .8
ridge = Ridge(alpha=.8)  
ridge.fit(Xtrain_scaled, ytrain)
y_pred_ridge_train = ridge.predict(Xtrain_scaled)
y_pred_ridge_test = ridge.predict(Xtest_scaled)


r2_train = r2_score(ytrain, y_pred_ridge_train)
r2_test = r2_score(ytest, y_pred_ridge_test)


print("Intercepto",ridge.intercept_,"Coeficientes",ridge.coef_, "R2 Train",r2_train,"R2 Test",r2_test )

Intercepto 17.433333333333334 Coeficientes [-1.87361571 -1.39992258  1.30732255  1.81156932 -0.68552688  1.38448781
 -0.02635773  1.46043045 -1.41053881 -0.79954933] R2 Train 0.9452638793953312 R2 Test 0.5612475485912081


In [50]:
# Lambda: .1
ridge = Ridge(alpha=.1)  
ridge.fit(Xtrain_scaled, ytrain)
y_pred_ridge_train = ridge.predict(Xtrain_scaled)
y_pred_ridge_test = ridge.predict(Xtest_scaled)


r2_train = r2_score(ytrain, y_pred_ridge_train)
r2_test = r2_score(ytest, y_pred_ridge_test)


print("Intercepto",ridge.intercept_,"Coeficientes",ridge.coef_, "R2 Train",r2_train,"R2 Test",r2_test )

Intercepto 17.43333333333333 Coeficientes [-2.07112439 -2.90605012  4.04654138  3.04446081 -0.0214042   2.34685586
 -0.24193943  2.00723997 -3.10258427 -1.39667525] R2 Train 0.9805234796986707 R2 Test 0.23411656981043372


Lambda = 0.1 (muy baja) los coeficientes son grandes, el modelo ajusta casi perfectamente el entrenamiento (R2 = 0.98), pero falla mucho en test (R2 = 0.23). Esto indica sobreajuste.

Lambda = 0.7 (intermedia) los coeficientes se reducen, el ajuste en train baja un poco (R2 = 0.95), pero la predicción en test mejora significativamente (R2 = 0.54). Aquí hay un mejor equilibrio entre ajustar y generalizar.

**_______________________________________________________________________________________________________________________**

### Regresión dummies (y = mpg)

In [24]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
df = df.drop(columns=["model"])
X = df.drop(columns=["mpg"])
y = df["mpg"]

In [25]:
X=pd.get_dummies(X, columns=['cyl','gear','carb'])
X = X.astype(int)

In [26]:
modelo=LinearRegression()
modelo.fit(X,y)
y_pred= modelo.predict(X)
r2=r2_score(y,y_pred)
print(modelo.coef_,modelo.intercept_, "R2",r2)

[ 0.01830887 -0.07862013  1.96436269 -2.60198229  0.38957261  2.02955372
  0.64184981  0.00967623 -1.82438515  1.81470893 -1.72580159 -1.1309654
  2.856767   -1.55689123 -2.61736311 -0.70136616 -0.57998694  0.54005481
  4.91555263] 22.056737406563265 R2 0.8976755550941543


Se hizo una regresión lineal para predecir el consumo de combustible (mpg) usando las características del auto y convirtiendo algunas variables en categorías (dummies). El modelo explica casi el 90% de la variación en mpg, lo que significa que ajusta bastante bien. El intercepto indica el valor base de mpg, y los coeficientes muestran cómo cada característica afecta el consumo: los positivos aumentan el mpg y los negativos lo reducen. En pocas palabras, el modelo captura bien cómo las distintas propiedades del auto influyen en su rendimiento de combustible.

### Train - Test 

In [29]:
Xtrain, Xtest, ytrain, ytest=train_test_split(X,y,train_size=.4,random_state=137)

In [30]:
modelo_dummies = LinearRegression()
modelo_dummies.fit(Xtrain, ytrain)
y_pred_train = modelo_dummies.predict(Xtrain)
y_pred_test = modelo_dummies.predict(Xtest)

print("R2 train:", r2_score(ytrain, y_pred_train))
print("R2 test:",  r2_score(ytest, y_pred_test))

R2 train: 1.0
R2 test: 0.28303765950187


El R2 de entrenamiento es 1.0, lo que indica que el modelo ajusta perfectamente los datos de entrenamiento, el de prueba = 0.28 indica sobreajuste. Usar dummies con pocos datos de entrenamiento hace que el modelo se adapte demasiado a esos datos y pierda capacidad de predecir correctamente en nuevas observaciones.

**_______________________________________________________________**

### Regresión Dummies (y = qsec)

In [38]:
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
df = df.drop(columns=["model"])
X = df.drop(columns=["qsec"])
y = df["qsec"]

In [39]:
X=pd.get_dummies(X, columns=['cyl','gear','carb'])
X = X.astype(int)

In [40]:
modelo=LinearRegression()
modelo.fit(X,y)
y_pred= modelo.predict(X)
r2=r2_score(y,y_pred)
print(modelo.coef_,modelo.intercept_, "R2",r2)

[ 0.01977242  0.00654616 -0.0022363  -0.12178602  0.41294793  0.31242369
 -1.63412266  1.56234142  0.22363696 -1.78597838 -0.4515632   0.98504616
 -0.53348296  0.65955613 -0.09818823  0.8623446  -1.00248583 -0.26679561
 -0.15443106] 16.31671033243791 R2 0.9057905221872551


Logra un R2 de 0.91, lo que significa que explica aproximadamente el 90% de la variabilidad de qsec en los datos. Los coeficientes indican cómo cada variable afecta a qsec: valores positivos lo aumentan y valores negativos lo disminuyen

### Train - Test

In [41]:
Xtrain, Xtest, ytrain, ytest=train_test_split(X,y,train_size=.4,random_state=137)

In [42]:
modelo_dummies = LinearRegression()
modelo_dummies.fit(Xtrain, ytrain)
y_pred_train = modelo_dummies.predict(Xtrain)
y_pred_test = modelo_dummies.predict(Xtest)

print("R2 train:", r2_score(ytrain, y_pred_train))
print("R2 test:",  r2_score(ytest, y_pred_test))

R2 train: 1.0
R2 test: -0.22029889608534536


El modelo ajusta perfectamente los datos de entrenamiento (R2 = 1.0), pero no generaliza nada bien a los datos de prueba (R2 negativo, -0.22). Esto indica un sobreajuste extremo, el modelo memoriza los datos de entrenamiento en lugar de aprender patrones generales, por lo que falla completamente al predecir qsec en datos nuevos.

### Comparación

Para mpg, comparar el modelo sin dummies con el modelo con dummies muestra que incluir las variables categóricas transforma un poco mejor el ajuste: el R2 sube de aproximadamente 0.87–0.90 a 0.898. Esto significa que el modelo con dummies explica un poco más la variabilidad de mpg, aunque la mejora no es muy grande porque el dataset es pequeño y algunas categorías tienen pocas observaciones.

Para qsec, la comparación es similar: el modelo sin dummies tiene un R2 de aproximadamente 0.875, mientras que con dummies sube a 0.906. Esto indica que agregar las variables categóricas ayuda a explicar mejor los tiempos de aceleración, aunque la diferencia sigue siendo moderada. En ambos casos, usar dummies mejora el modelo, pero no de manera dramática debido al tamaño y la estructura del dataset.